In [2]:
import pandas as pd
import altair as alt

In [3]:
#create base row of dates
dates = pd.DataFrame({'date':pd.date_range(start = '2020-02-01', 
              end = '2021-10-15', 
              freq='D')})

In [4]:
#case data
case_data = pd.read_csv("data_raw/RAW_us_confirmed_cases.csv")
case_df = case_data[case_data["FIPS"]==21111]
case_date_columns = case_df.columns[11:]
jefferson = case_df.melt(value_vars = case_date_columns,
                    var_name = "date",
                    value_name = "confirmed_cases_cumulative")
jefferson['confirmed_cases']=(jefferson['confirmed_cases_cumulative']-
                              jefferson['confirmed_cases_cumulative'].shift(1).fillna(0))
jefferson["7dconfirmed_cases"]= jefferson["confirmed_cases"].rolling(window = 7,).mean().fillna(0)
jefferson["date"] = pd.to_datetime(jefferson["date"])
jefferson['inc'] = jefferson["7dconfirmed_cases"] - jefferson["7dconfirmed_cases"].shift(1)

In [6]:
#mask mandates
mask_data = pd.read_csv("data_raw/U.S._State_and_Territorial_Public_Mask_Mandates_From_April_10__2020_through_August_15__2021_by_County_by_Day.csv")
mask_df = mask_data[(mask_data["State_Tribe_Territory"]=="KY")&
    (mask_data["County_Name"]=="Jefferson County")]
mask_df_cleaned = mask_df[["date","Face_Masks_Required_in_Public"]]
mask_df_cleaned["date"] = pd.to_datetime(mask_df_cleaned["date"])

<ipython-input-6-213b3c03bd18>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mask_df_cleaned["date"] = pd.to_datetime(mask_df_cleaned["date"])


In [7]:
data = pd.merge(dates,jefferson,
         on = "date",
         how = "left"
        )

In [8]:
data = pd.merge(data,mask_df_cleaned,
         on = "date",
         how = "left"
        )

In [15]:
data.to_csv("data_clean/mask_data_confirmed_cases_jefferson_ky.csv")

In [16]:
smooth_chart = alt.Chart(jefferson).mark_line(color = "grey",opacity = 0.5).encode(
    x = "date:T",
    y = "7dconfirmed_cases:Q",
    tooltip = ['date','confirmed_cases','day(date)']
    
)

In [17]:
masks = alt.Chart(data[data["Face_Masks_Required_in_Public"].isnull()==False],
                 title = "7 Day Rolling Average of Confirmed Covid Cases in Jefferson County").mark_line().encode(
    x=alt.X('date:T', axis=alt.Axis(title='Date')),
    y=alt.Y('7dconfirmed_cases:Q', axis=alt.Axis(title='7d Rolling Average Confirmed Covid Cases')),
    color = "Face_Masks_Required_in_Public:N",
    tooltip = ['date','confirmed_cases','day(date)']
    
)

In [18]:
(smooth_chart+masks)

alt.LayerChart(...)

In [19]:
smooth_chart_inc = alt.Chart(jefferson).mark_line(color = "grey",opacity = 0.5).encode(
    x = "date:T",
    y = "inc",
    tooltip = ['date','confirmed_cases','day(date)']
    
)

In [20]:
masks_inc = alt.Chart(data[data["Face_Masks_Required_in_Public"].isnull()==False],
                 title = "Inc in Confirmed Covid Cases in Jefferson County").mark_line().encode(
    x=alt.X('date:T', axis=alt.Axis(title='Date')),
    y=alt.Y('inc:Q', axis=alt.Axis(title='Inc vis -a - vis previous day cases')),
    color = "Face_Masks_Required_in_Public:N",
    tooltip = ['date','confirmed_cases','day(date)']
    
)

In [21]:
smooth_chart_inc + masks_inc

alt.LayerChart(...)

In [22]:
data[data["Face_Masks_Required_in_Public"].isnull()==False].iloc[335:]

,date,confirmed_cases_cumulative,confirmed_cases,7dconfirmed_cases,inc,Face_Masks_Required_in_Public
495,2021-06-10,82606,37.0,36.571429,-3.428571,Yes
496,2021-06-11,82637,31.0,29.857143,-6.714286,No
497,2021-06-12,82674,37.0,30.142857,0.285714,No
498,2021-06-13,82691,17.0,32.571429,2.428571,No
499,2021-06-14,82720,29.0,34.428571,1.857143,No
...,...,...,...,...,...,...
557,2021-08-11,88189,344.0,285.714286,8.000000,No
558,2021-08-12,88502,313.0,282.571429,-3.142857,No
559,2021-08-13,88986,484.0,306.857143,24.285714,No
560,2021-08-14,88986,0.0,306.857143,0.000000,No
